In [ ]:
import  csv
import  rdflib
from    rdflib import Graph, Namespace, URIRef, Literal, RDF, RDFS, OWL, URIRef
from    rdflib.namespace import FOAF, DCTERMS, XSD, RDF, SDO, RDFS

g = Graph()
foodnutrition = Namespace('http://kg-course/foodnutrition/')
# Define namespaces
g.bind("foodnutrition", foodnutrition)

g.parse("final.ttl", format="turtle")
print("Turtle file loaded. The graph has {} triples.".format(len(g)))


## 3.1 NLP extract ingredients

Preprocess Reviews.txt

In [ ]:
#clean Reviews.txt
import pandas as pd
import re

# Read the entire file as raw text
with open('Reviews_small.txt', 'r', encoding='utf-8') as file:
    raw_text = file.read()

# Replace triple quotes and newlines within reviews
# Keep newlines that separate proper reviews (i.e., before ReviewId)
cleaned_text = re.sub(r'(""""|\n)(?!(\d+\t))', ' ', raw_text)

# Now cleaned_text should have each review on a single line, except for proper row separations
from io import StringIO

# Use StringIO to treat the cleaned text like a file
cleaned_file = StringIO(cleaned_text)

# Load into pandas DataFrame
rev_df = pd.read_csv(cleaned_file, delimiter='\t', on_bad_lines='skip')

Use NLP to extract ingredients

In [ ]:
#### Without csv
import pandas as pd
from collections import defaultdict
from datetime import datetime
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import csv

# Load your DataFrame (assuming 'rev_df' is already prepared)
# rev_df = pd.read_csv('path_to_your_cleaned_file.csv')  # Uncomment if needed

# NER model for food
tokenizer_ner = AutoTokenizer.from_pretrained("Dizex/InstaFoodRoBERTa-NER")
model_ner = AutoModelForTokenClassification.from_pretrained("Dizex/InstaFoodRoBERTa-NER").to("cpu")
ner_pipe = pipeline("ner", model=model_ner, tokenizer=tokenizer_ner, device=0)  # device=-1 for CPU

# Function to convert entities to a list
def convert_entities_to_list(text, entities):
    ents = []
    for ent in entities:
        e = {"start": ent["start"], "end": ent["end"], "label": ent["entity_group"]}
        if ents and -1 <= ent["start"] - ents[-1]["end"] <= 1 and ents[-1]["label"] == e["label"]:
            ents[-1]["end"] = e["end"]
            continue
        ents.append(e)
    return [text[e["start"]:e["end"]] for e in ents]

# Initialize storage for ingredient counts
ingredient_counts = defaultdict(int)
ingredients_list = []  # Store extracted ingredients for DataFrame

# Process each review
for idx, row in rev_df.iterrows():
    review_text = row.get('Review', '')  # Ensure the column name matches your DataFrame

    # Run NER model
    ner_entity_results = ner_pipe(review_text, aggregation_strategy="simple")
    ingredients = convert_entities_to_list(review_text, ner_entity_results)

    # Update ingredient counts
    for recipeName in ingredients:
        ingredient_counts[recipeName] += 1

    # Store extracted ingredients for DataFrame
    ingredients_list.append(', '.join(ingredients) if ingredients else 'None')

# Add new "Ingredients" column to rev_df
rev_df['Ingredients'] = ingredients_list

# Generate timestamps for unique filenames
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
extracted_file = f"{current_time}_extracted_ingredientd:\Chrome downloads\Description_of_the_Assignment.pdfs.csv"
counted_file = f"{current_time}_ingredient_counts.csv"

# Save updated DataFrame to CSV (optional)
rev_df.to_csv(extracted_file, index=False, encoding='utf-8')
print(f"Updated DataFrame with extracted ingredients saved to {extracted_file}")

# Write ingredient counts to a separate CSV file
with open(counted_file, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Ingredient', 'Count']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for recipeName, count in sorted(ingredient_counts.items(), key=lambda x: x[1], reverse=True):
        writer.writerow({'Ingredient': recipeName, 'Count': count})

print(f"Ingredient counts saved to {counted_file}")


Add extracted ingredients to our KG

In [ ]:
## Add ingredients to Review class
# Add Review Properties for extracted ingredients
extr_ingred = foodnutrition["extractedIngredients"]

g.add((extr_ingred, RDFS.domain, SDO.Review))
g.add((extr_ingred, RDFS.range, RDFS.Literal))


for idx, row in rev_df.iterrows():
    review_uri = foodnutrition[f"review_{row['ReviewId']}"]  # Unique review identifier
    g.add((review_uri, RDF.type, SDO.Review))  # Add review instance

    # Iterate over individual ingredients and add separately
    for recipeName in row["Ingredients"].split(", "):  # Assuming ingredients are comma-separated
        g.add((review_uri, extr_ingred, Literal(recipeName)))  # Add each ingredient as a separate triple


# Save the updated KG to a file (optional)
g.serialize("updated_kg.ttl", format="turtle")

print("Ingredients added to the Knowledge Graph!")


## 3.2 NLP Sentiment analysis on Reviews.txt

Preprocess Reviews.txt

In [ ]:
import pandas as pd
# Read the entire file as raw text
with open('Reviews_small.txt', 'r', encoding='utf-8') as file:
    raw_text = file.read()

import re

# Replace triple quotes and newlines within reviews
# Keep newlines that separate proper reviews (i.e., before ReviewId)
cleaned_text = re.sub(r'(""""|\n)(?!(\d+\t))', ' ', raw_text)

# Now cleaned_text should have each review on a single line, except for proper row separations
from io import StringIO

# Use StringIO to treat the cleaned text like a file
cleaned_file = StringIO(cleaned_text)

# Load into pandas DataFrame
rev_df = pd.read_csv(cleaned_file, delimiter='\t', on_bad_lines='skip')

NLP extract sentiment and confidence

In [ ]:
import logging
logging.getLogger("transformers").setLevel(logging.WARNING)

# Load Reviews.txt (assuming tab-separated values)
# reviews_df = pd.read_csv('Reviews_test.txt', delimiter='\t')
reviews_df = rev_df

from transformers import pipeline

# Load the sentiment analysis pipeline and use cuda device
sentiment_pipeline = pipeline("sentiment-analysis", device=0)
# sentiment_pipeline = pipeline("sentiment-analysis", model="facebook/bart-large-mnli", device=0) # for neutral sentiment

# Handle NaN values in the 'Review' column
reviews_df['Review'] = reviews_df['Review'].fillna("")

# Apply sentiment analysis to the reviews
sentiment_results = sentiment_pipeline(reviews_df['Review'].tolist())

# Add sentiment results back to DataFrame
reviews_df['Sentiment'] = [result['label'] for result in sentiment_results]
reviews_df['Confidence'] = [result['score'] for result in sentiment_results]

# Preview the results
# print(reviews_df[['ReviewId', 'Review', 'Sentiment', 'Confidence']])

# # Save the results to a CSV file
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"{current_time}_extracted_ingredients.csv"
reviews_df[['ReviewId', 'RecipeId', 'Review', 'Sentiment', 'Confidence']].to_csv(output_file, index=False)


Add data from Reviews to our KG

In [ ]:
# Define sentiment and confidence properties for the Review class
sentiment = foodnutrition["sentiment"]
confidence = foodnutrition["confidence"]

g.add((sentiment, RDFS.domain, SDO.Review))
g.add((sentiment, RDFS.range, RDFS.Literal))

g.add((confidence, RDFS.domain, SDO.Review))
g.add((confidence, RDFS.range, RDFS.Literal))


i = 0
# Loop through each review and add it to the KG
for index, row in reviews_df.iterrows():
    i += 1
    
    # Create a unique URI for each review
    # review_uri = foodnutrition["Review/" + str(i)]
    review_uri = foodnutrition[f"Review/{row['ReviewId']}"]

    # Add the Review type to the graph
    g.add((review_uri, RDF.type, SDO.Review))
    # Add properties to the Review
    g.set((review_uri, foodnutrition.reviewId, Literal(row['ReviewId'])))
    g.set((review_uri, foodnutrition.recipeID, Literal(row['RecipeId'])))
    g.set((review_uri, foodnutrition.authorId, Literal(row['AuthorId'])))
    g.set((review_uri, SDO.author, Literal(row['AuthorName'])))
    g.set((review_uri, SDO.reviewBody, Literal(row['Review'])))
    g.set((review_uri, SDO.datePublished, Literal(row['DateSubmitted'], datatype=XSD.date)))
    g.set((review_uri, SDO.dateModified, Literal(row['DateModified'], datatype=XSD.date)))
    
    # Add sentiment and confidence
    g.set((review_uri, foodnutrition.sentiment, Literal(row['Sentiment'], datatype=XSD.string)))
    g.set((review_uri, foodnutrition.confidence, Literal(row['Confidence'], datatype=XSD.float)))

    if i == 10: break

print()



In [ ]:
g.serialize('finalUnstructAndStruct.ttl')

## 3.3 Entity linking

In [ ]:
# Entity linking and ontology alignment – identify and integrate frequently mentioned ingredients by aligning them with existing ontology classes using OWL or SKOS mappings
import pandas as pd

# Convert ingredient_counts dictionary to a sorted DataFrame
top_5_ingredients = sorted(ingredient_counts.items(), key=lambda x: x[1], reverse=True)[:5]

# Create DataFrame
top_5_df = pd.DataFrame(top_5_ingredients, columns=['Ingredient', 'Count'])

# Display DataFrame
print(top_5_df)


In [ ]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

# SPARQL endpoints
WIKIDATA_SPARQL = "https://query.wikidata.org/sparql"

def query_wikidata(ingredient):
    """Query Wikidata to find an entity with the given label."""
    sparql = SPARQLWrapper(WIKIDATA_SPARQL)
    sparql.addCustomHttpHeader("User-Agent", "MyFoodOntologyBot/1.0 (your@email.com)")  # Required header

    query = f"""
    SELECT ?concept WHERE {{
      ?concept rdfs:label "{ingredient}"@en .
    }}
    LIMIT 1
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    try:
        results = sparql.query().convert()
        if results["results"]["bindings"]:
            return results["results"]["bindings"][0]["concept"]["value"]  # Return first match
    except Exception as e:
        print(f"Wikidata Query Error for {ingredient}: {e}")
    return None


# Create a new DataFrame to store mappings
ontology_mappings = []

for index, row in top_5_df.iterrows():
    recipeName = row['Ingredient']  # Get ingredient from top_5_df
    wikidata_match = query_wikidata(recipeName)

    ontology_mappings.append({
        "Ingredient": recipeName,
        "Wikidata_URI": wikidata_match
    })

# Convert to DataFrame
ontology_df = pd.DataFrame(ontology_mappings)

# Display results
print(ontology_df)


In [ ]:
from rdflib import URIRef, Literal
from rdflib.namespace import RDF, OWL, SKOS

# Define namespaces for your Knowledge Graph
wikidata = Namespace("http://www.wikidata.org/entity/")


# Example: Add mappings for the top 5 ingredients to the Knowledge Graph
for index, row in ontology_df.iterrows():
    ingredient_name = row["Ingredient"]
    wikidata_uri = row["Wikidata_URI"]
    
    # Create a unique URI for the ingredient in the KG
    ingredient_uri = foodnutrition[ingredient_name.replace(" ", "_")]  # Replace spaces with underscores to form valid URIs
    
    # Create RDF triples linking the ingredient to its Wikidata URI
    if wikidata_uri:
        # Add the ingredient as an instance of Ingredient class
        g.add((ingredient_uri, RDF.type, foodnutrition.Ingredient))
        
        # Link to the Wikidata URI using skos:exactMatch
        g.add((ingredient_uri, SKOS.exactMatch, URIRef(wikidata_uri)))
        
        print(f"Added {ingredient_name} with Wikidata URI: {wikidata_uri} and optional mappings.")
    else:
        print(f"No Wikidata URI found for {ingredient_name}")

# Optionally, serialize the updated Knowledge Graph with mappings
g.serialize("updated_foodnutrition_with_wikidata_uris.ttl", format="turtle")


## 3.4 Integrate external KGs 

In [ ]:
import  rdflib
from    rdflib import Graph, Namespace, URIRef, Literal, RDF, RDFS, OWL, Literal, URIRef
from    rdflib.namespace import FOAF, DCTERMS, XSD, RDF, SDO, RDFS
import pyvis
from pyvis.network import Network
from IPython.display import IFrame

# 1. Load the Turtle file into an RDF graph.
# Replace 'your_file.ttl' with the path to your Turtle file.
g = Graph()

g.parse("top1000ResNutRec.ttl", format="turtle")
print("Turtle file loaded. The graph has {} triples.".format(len(g)))

In [ ]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

# SPARQL endpoints
DBPEDIA_SPARQL = "https://dbpedia.org/sparql"
WIKIDATA_SPARQL = "https://query.wikidata.org/sparql"

def query_wikidata(recipeName):
    """Query Wikidata to find an entity with the given label."""
    sparql = SPARQLWrapper(WIKIDATA_SPARQL)
    sparql.addCustomHttpHeader("User-Agent", "MyFoodOntologyBot/1.0 (your@email.com)")  # Required header

    query = f"""
    SELECT  ?cusine WHERE{{
              ?dish wdt:P31 wd:Q746549;
              wdt:P2012 ?cusineID;
              rdfs:label ?label.
            ?cusineID rdfs:label ?cusine.
            filter contains("{recipeName}"@en, ?label)
    }}
            LIMIT 1
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    

    try:
        results = sparql.query().convert()
        if results["results"]["bindings"]:
            return results["results"]["bindings"][0]["cusine"]["value"]  # Return first match
    except Exception as e:
        print(f"Wikidata Query Error for {recipeName}: {e}")
    return None

query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbd:	<http://dbpedia.org/datatype/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?name ?recipe
WHERE {
    ?recipe a schema:Recipe ;
      schema:name ?name .
}
"""
qResult = g.query(query)

for row in qResult:
    cusine = query_wikidata(row.name)
    if cusine is not None:
        g.add((row.recipe, SDO.servesCuisine, Literal(cusine)))
        print(f" {row.name}  + {cusine}")

In [ ]:
g.serialize('finalWithEntityLink.ttl')